In [ ]:
import pandas as pd

dataset = pd.read_csv("Copyphishing.csv")

cleaned_dataset = dataset[['Feature', 'Boolean']].dropna()
cleaned_dataset = cleaned_dataset.rename(columns={'Feature': 'text', 'Boolean': 'label'})

cleaned_dataset['label'] = cleaned_dataset['label'].astype(int)

print(cleaned_dataset.head())

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset

# loading BERT Tokenizer...
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

dataset = Dataset.from_pandas(cleaned_dataset)

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# applying tokenization on each batch
tokenized_dataset = dataset.map(tokenize_function, batched=True)

print("Tokenized Data : \n")
print(tokenized_dataset[0])

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import torch

# training -> 80%, validation -> 20%
train_dataset, val_dataset = tokenized_dataset.train_test_split(test_size=0.2).values()

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",       
    evaluation_strategy="epoch",  
    learning_rate=3e-5,           
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    num_train_epochs=5,           
    weight_decay=0.01,            
    logging_dir="./logs",         
    logging_steps=10,             
    save_steps=500,               
    report_to="none"
)

# initializing ...
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  
)

trainer.train()

evaluation_results = trainer.evaluate()
print("Evaluation Metrics:", evaluation_results)

In [ ]:
import optuna # optuna is a open-source hyperparameter optimization framework
from transformers import Trainer, TrainingArguments

def model_training(trial):

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_epochs = trial.suggest_int("num_epochs", 3, 6)

    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        num_train_epochs=num_epochs,
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
    )
    
    trainer.train()
    evaluation_results = trainer.evaluate()
    return evaluation_results["eval_f1"]

# Running Optuna study ...
study = optuna.create_study(direction="maximize")
study.optimize(model_training, n_trials=10)

print("Best Hyperparameters:", study.best_params)